## 英德机器翻译 
模型：transformer
数据：Multi30k
1. https://www.statmt.org/wmt16/multimodal-task.html#task1
2. [torchtext.datasets.Multi30k](https://pytorch.org/text/stable/datasets.html#multi30k)

###### 下载安装依赖
1. torchdata、spacy
2. 语言模型工具包
```python
pip install -U torchdata
pip install -U spacy
python -m spacy download en_core_web_sm
python -m spacy download de_core_news_sm

#### 下载数据

In [2]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torchtext.datasets import multi30k, Multi30k
from typing import Iterable, List


# We need to modify the URLs for the dataset since the links to the original dataset are broken
# Refer to https://github.com/pytorch/text/issues/1756#issuecomment-1163664163 for more info
multi30k.URL["train"] = "https://raw.githubusercontent.com/neychev/small_DL_repo/master/datasets/Multi30k/training.tar.gz"
multi30k.URL["valid"] = "https://raw.githubusercontent.com/neychev/small_DL_repo/master/datasets/Multi30k/validation.tar.gz"

SRC_LANGUAGE = 'de'
TGT_LANGUAGE = 'en'

# 两种语言里的数据，key是'de'|'en',value是模型工具
token_transform = {}
# 两种语言的词表，key是'de'|'en',value是vocab
vocab_transform = {}

# 加载spacy的模型工具
token_transform[SRC_LANGUAGE] = get_tokenizer('spacy', language='de_core_news_sm')
token_transform[TGT_LANGUAGE] = get_tokenizer('spacy', language='en_core_web_sm')


# helper function to yield list of tokens
def yield_tokens(data_iter: Iterable, language: str) -> List[str]:
    language_index = {SRC_LANGUAGE: 0, TGT_LANGUAGE: 1}

    for data_sample in data_iter:
        # 文本分词为tokens
        yield token_transform[language](data_sample[language_index[language]])

# 定义特殊token的索引：未知、填充、开始、结束
UNK_IDX, PAD_IDX, BOS_IDX, EOS_IDX = 0, 1, 2, 3
# 特殊的token，注意位置顺序
special_symbols = ['<unk>', '<pad>', '<bos>', '<eos>']

for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
    # 训练数据
    train_iter = Multi30k(root='./../data/', split='train', language_pair=(SRC_LANGUAGE, TGT_LANGUAGE))
    # 生成两种语言的词表
    vocab_transform[ln] = build_vocab_from_iterator(yield_tokens(train_iter, ln),
                                                    min_freq=1,
                                                    specials=special_symbols, #加入特殊token
                                                    special_first=True)

# 设置默认token的索引：UNK_IDX
# 作用是在输入token在词表里找不到的时候，返回的默认索引
# 如果不设置默认token，那么当找不到token，会抛出`RuntimeError`错误
for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
  vocab_transform[ln].set_default_index(UNK_IDX)

### 再次预处理数据
1. 增加开始和结束的token
2. 截断和填充

注意：torch.nn.utils.rnn.pad_sequence只会以最长序列为固定序列，进行填充，不截断

In [2]:
from torch.nn.utils.rnn import pad_sequence

# helper function to club together sequential operations
def sequential_transforms(*transforms):
    def func(txt_input):
        for transform in transforms:
            txt_input = transform(txt_input)
        return txt_input
    return func

# 添加开始和结束token
# token_ids:一个序列中token索引组成的list
def tensor_transform(token_ids: List[int]):
    return torch.cat((torch.tensor([BOS_IDX]),
                      torch.tensor(token_ids),
                      torch.tensor([EOS_IDX])))



# 两种语言的处理函数：文本token化，词典输出indice,添加开始和结束token
text_transform = {}
for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
    text_transform[ln] = sequential_transforms(token_transform[ln], #token工具
                                               vocab_transform[ln], #词典
                                               tensor_transform) # Add BOS/EOS and create tensor



#torch.utils.data.DataLoader(dataset, 
#                         batch_size=1, shuffle=None, sampler=None, batch_sampler=None, 
#                         num_workers=0, collate_fn=None, pin_memory=False, drop_last=False, 
#                         timeout=0, worker_init_fn=None, multiprocessing_context=None, generator=None, *, 
#                         prefetch_factor=None, persistent_workers=False, pin_memory_device='')
def collate_fn(batch):
    """
    torch.utils.data.DataLoader的分配函数

    Arguments:
        batch: 是对DataLoader中的dataset遍历，batch个遍历数据

    Returns:
        src_batch, Tensor of shape ``[max_sequence，batch_size]``
        tgt_batch, Tensor of shape ``[max_sequence，batch_size]``
    
    """
    src_batch, tgt_batch = [], []
    for src_sample, tgt_sample in batch:
        #去除字符串末尾的换行符，输出：str -> [toekn,...] -> [index,...] -> [2,index,..,3]
        src_batch.append(text_transform[SRC_LANGUAGE](src_sample.rstrip("\n")))
        tgt_batch.append(text_transform[TGT_LANGUAGE](tgt_sample.rstrip("\n")))

    # src_batch, tgt_batch都是2d list
    # pad_sequence只填充不截断
    # 不过这里pad_sequence的参数batch_first默认是False
    # 所以src_batch和tgt_batch形状是(max_sequence，batch_size)
    src_batch = pad_sequence(src_batch, padding_value=PAD_IDX)
    tgt_batch = pad_sequence(tgt_batch, padding_value=PAD_IDX)

    return src_batch, tgt_batch

### 构建网络

In [3]:
from torch import Tensor
import torch
import torch.nn as nn
from torch.nn import Transformer
import math
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 位置编码矩阵self.pos_embedding
class PositionalEncoding(nn.Module):
    def __init__(self,
                 emb_size: int,
                 dropout: float,
                 maxlen: int = 5000):
        super(PositionalEncoding, self).__init__()
        den = torch.exp(- torch.arange(0, emb_size, 2)* math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        #形状是(max_len, emb_size)
        pos_embedding = torch.zeros((maxlen, emb_size))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        #形状变成：(max_len, 1, emb_size)
        pos_embedding = pos_embedding.unsqueeze(-2)

        self.dropout = nn.Dropout(dropout)
        self.register_buffer('pos_embedding', pos_embedding)

    def forward(self, token_embedding: Tensor):
        """
        self.pos_embedding的形状是(max_len, 1, emb_size)
        和输入张量token_embedding相加会有广播机制，token_embedding比如是3D张量

        Arguments:
            token_embedding: Tensor, shape ``[seq_len, batch_size, emb_size]``
        """
        return self.dropout(token_embedding + self.pos_embedding[:token_embedding.size(0), :])

# 嵌入层
class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size: int, emb_size):
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size

    def forward(self, tokens: Tensor):
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)

# Seq2Seq Network
class Seq2SeqTransformer(nn.Module):
    def __init__(self,
                 num_encoder_layers: int,
                 num_decoder_layers: int,
                 emb_size: int,
                 nhead: int,
                 src_vocab_size: int,
                 tgt_vocab_size: int,
                 dim_feedforward: int = 512,
                 dropout: float = 0.1):
        super(Seq2SeqTransformer, self).__init__()
        self.transformer = Transformer(d_model=emb_size, #输入维度
                                       nhead=nhead,
                                       num_encoder_layers=num_encoder_layers,
                                       num_decoder_layers=num_decoder_layers,
                                       dim_feedforward=dim_feedforward, #前馈网络维度
                                       dropout=dropout)
        #输出层                               
        self.generator = nn.Linear(emb_size, tgt_vocab_size)
        #嵌入层
        self.src_tok_emb = TokenEmbedding(src_vocab_size, emb_size)
        self.tgt_tok_emb = TokenEmbedding(tgt_vocab_size, emb_size)
        #位置编码
        self.positional_encoding = PositionalEncoding(
            emb_size, dropout=dropout)

    def forward(self,
                src: Tensor,
                trg: Tensor,
                src_mask: Tensor,
                tgt_mask: Tensor,
                src_padding_mask: Tensor,
                tgt_padding_mask: Tensor,
                memory_key_padding_mask: Tensor):
        """
        Arguments:
            src: Tensor, shape ``[src_seq_len, batch_size]``
            trg: Tensor, shape ``[trg_seq_len, batch_size]``

            src_mask: Tensor, shape ``[src_seq_len, src_seq_len]``,None,编码器不需要屏蔽未来位置
            tgt_mask: Tensor, shape ``[tgt_seq_len, tgt_seq_len]``
            src_padding_mask: Tensor, shape ``[batch_size, src_seq_len]``
            tgt_padding_mask: Tensor, shape ``[batch_size, tgt_seq_len]``

            memory_key_padding_mask: Tensor, 
            编码器提取特征作为编码器注意力机制的key和value，它的填充掩码
            一般来说等于源语言的填充掩码，也就是编码器的填充掩码

        Returns:
            output Tensor of shape ``[tgt_seq_len, batch_size, tgt_vocab_size]``
        """
        src_emb = self.positional_encoding(self.src_tok_emb(src))
        tgt_emb = self.positional_encoding(self.tgt_tok_emb(trg))
        outs = self.transformer(src_emb, tgt_emb, src_mask, tgt_mask, None,
                                src_padding_mask, tgt_padding_mask, memory_key_padding_mask)
        return self.generator(outs)

    #预测准备
    #编码器前面传递
    def encode(self, src: Tensor, src_mask: Tensor):
        """
        Arguments:
            src: Tensor, shape ``[src_seq_len, batch_size]``
            src_mask: Tensor, shape ``[src_seq_len, src_seq_len]``,注意编码器不需要屏蔽未来位置

        Returns:
            output Tensor of shape ``[src_seq_len, batch_size, emb_size]``
        """
        return self.transformer.encoder(self.positional_encoding(
                            self.src_tok_emb(src)), src_mask)

    #解码器前向传递
    def decode(self, tgt: Tensor, memory: Tensor, tgt_mask: Tensor):
        """
        Arguments:
            tgt: Tensor, shape ``[trg_seq_len, batch_size]``

            tgt_mask: Tensor, shape ``[tgt_seq_len, tgt_seq_len]``
            memory: Tensor, shape

        Returns:
            output Tensor of shape ``[tgt_seq_len, batch_size, emb_size]``
        """
        return self.transformer.decoder(self.positional_encoding(
                          self.tgt_tok_emb(tgt)), memory,
                          tgt_mask)

#### 两种掩码配置

In [4]:
def generate_square_subsequent_mask(sz):
    """
    创建屏蔽未来位置的掩码（浮点数掩码）
    返回一个上三角矩阵，元素取值为0代表不屏蔽，-inf是负无穷，代表屏蔽

    Arguments:
        sz:int,序列长度

    Returns:
        output Tensor of shape ``[sz, sz]``
    """
    mask = (torch.triu(torch.ones((sz, sz), device=DEVICE)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask


def create_mask(src, tgt):
    """
    1. 调用创建屏蔽未来位置掩码的函数
    2. 创建填充掩码

    Arguments:
        src: Tensor, shape ``[src_seq_len, batch_size]``源语言张量
        tgt: Tensor, shape ``[trg_seq_len, batch_size]``目标语言张量

    Returns:
        src_mask, Tensor, shape ``[src_seq_len, src_seq_len]``
        tgt_mask, Tensor, shape ``[trg_seq_len, trg_seq_len]``

        src_padding_mask: Tensor, shape ``[batch_size, src_seq_len]``
        tgt_padding_mask: Tensor, shape ``[batch_size, tgt_seq_len]``
    """
    
    src_seq_len = src.shape[0]
    tgt_seq_len = tgt.shape[0]

    tgt_mask = generate_square_subsequent_mask(tgt_seq_len)
    # 源语言是输入给编码器，不需要屏蔽未来位置，所以所有元素是False
    src_mask = torch.zeros((src_seq_len, src_seq_len),device=DEVICE).type(torch.bool)

    #很简单得了逻辑：位置是PAD_IDX设置为True,代表需要屏蔽
    src_padding_mask = (src == PAD_IDX).transpose(0, 1)
    tgt_padding_mask = (tgt == PAD_IDX).transpose(0, 1)
    return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask


### 实例化模型

In [5]:
torch.manual_seed(0)

# 两种语言的词典大小
SRC_VOCAB_SIZE = len(vocab_transform[SRC_LANGUAGE])
TGT_VOCAB_SIZE = len(vocab_transform[TGT_LANGUAGE])

EMB_SIZE = 512            #transformer输出维度
NHEAD = 8                 #注意力头数量
FFN_HID_DIM = 512         #前馈网络隐藏层维度
BATCH_SIZE = 128          #批量大小
NUM_ENCODER_LAYERS = 3    #transformer的编码器层数
NUM_DECODER_LAYERS = 3    #transformer的解码器层数

transformer = Seq2SeqTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE,
                                 NHEAD, SRC_VOCAB_SIZE, TGT_VOCAB_SIZE, FFN_HID_DIM)

#初始化参数
for p in transformer.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)

transformer = transformer.to(DEVICE)

#损失函数：交叉熵
loss_fn = torch.nn.CrossEntropyLoss(ignore_index=PAD_IDX)

optimizer = torch.optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

d:\anaconda3\Lib\site-packages\torch\nn\modules\transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


### 训练和评估

In [6]:
from torch.utils.data import DataLoader

def train_epoch(model, optimizer):
    model.train()
    losses = 0
    train_iter = Multi30k(split='train', language_pair=(SRC_LANGUAGE, TGT_LANGUAGE))
    
    train_dataloader = DataLoader(train_iter, batch_size=BATCH_SIZE, collate_fn=collate_fn)

    for src, tgt in train_dataloader:
        # src, tgt形状都是(sequence,batch_size)
        src = src.to(DEVICE)
        tgt = tgt.to(DEVICE)

        # 输入去掉最后一个token
        tgt_input = tgt[:-1, :]

        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

        #(sequence, batch_size, vocab_size)
        logits = model(src, tgt_input, src_mask, tgt_mask,src_padding_mask, tgt_padding_mask, src_padding_mask)

        optimizer.zero_grad()

        #标签向右移动一位
        tgt_out = tgt[1:, :]
        #logit形状变为：(sequence*batch_size, vocab_size)
        #tgt_out形状变为：(sequence*batch_size,)
        #损失函数函数在内部会对输入进行softmax操作
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        loss.backward()

        optimizer.step()
        losses += loss.item()

    return losses / len(list(train_dataloader))


def evaluate(model):
    model.eval()
    losses = 0

    val_iter = Multi30k(split='valid', language_pair=(SRC_LANGUAGE, TGT_LANGUAGE))
    val_dataloader = DataLoader(val_iter, batch_size=BATCH_SIZE, collate_fn=collate_fn)

    for src, tgt in val_dataloader:
        src = src.to(DEVICE)
        tgt = tgt.to(DEVICE)

        tgt_input = tgt[:-1, :]

        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

        logits = model(src, tgt_input, src_mask, tgt_mask,src_padding_mask, tgt_padding_mask, src_padding_mask)

        tgt_out = tgt[1:, :]
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        losses += loss.item()

    return losses / len(list(val_dataloader))


#训练
from timeit import default_timer as timer
NUM_EPOCHS = 18

for epoch in range(1, NUM_EPOCHS+1):
    start_time = timer()
    train_loss = train_epoch(transformer, optimizer)
    end_time = timer()
    val_loss = evaluate(transformer)
    print((f"Epoch: {epoch}, Train loss: {train_loss:.3f}, Val loss: {val_loss:.3f}, "f"Epoch time = {(end_time - start_time):.3f}s"))

d:\anaconda3\Lib\site-packages\torch\nn\functional.py:5076: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(
d:\anaconda3\Lib\site-packages\torch\utils\data\datapipes\iter\combining.py:333: UserWarning: Some child DataPipes are not exhausted when __iter__ is called. We are resetting the buffer and each child DataPipe will read from the start again.
  warnings.warn("Some child DataPipes are not exhausted when __iter__ is called. We are resetting "


Epoch: 1, Train loss: 5.342, Val loss: 4.105, Epoch time = 21.515s
Epoch: 2, Train loss: 3.761, Val loss: 3.308, Epoch time = 17.248s
Epoch: 3, Train loss: 3.157, Val loss: 2.885, Epoch time = 17.294s
Epoch: 4, Train loss: 2.764, Val loss: 2.626, Epoch time = 17.216s
Epoch: 5, Train loss: 2.474, Val loss: 2.459, Epoch time = 17.352s
Epoch: 6, Train loss: 2.246, Val loss: 2.312, Epoch time = 17.321s
Epoch: 7, Train loss: 2.055, Val loss: 2.202, Epoch time = 17.283s
Epoch: 8, Train loss: 1.893, Val loss: 2.109, Epoch time = 17.310s
Epoch: 9, Train loss: 1.753, Val loss: 2.050, Epoch time = 17.298s
Epoch: 10, Train loss: 1.634, Val loss: 2.002, Epoch time = 17.358s
Epoch: 11, Train loss: 1.519, Val loss: 1.972, Epoch time = 17.389s
Epoch: 12, Train loss: 1.418, Val loss: 1.964, Epoch time = 17.349s
Epoch: 13, Train loss: 1.330, Val loss: 1.969, Epoch time = 17.222s
Epoch: 14, Train loss: 1.250, Val loss: 1.929, Epoch time = 17.386s
Epoch: 15, Train loss: 1.172, Val loss: 1.917, Epoch time

### 翻译

In [7]:
# 预测时解码器的处理
def greedy_decode(model, src, src_mask, max_len, start_symbol):
    """
    1. src输入给编码器，作为解码器注意力机制中的特征
    2. 以start_symbol作为开始的token构建解码器的初始输入序列，batch和sequence都是1
    3. 依旧要根据sequence长度创建屏蔽未来的掩码
    4. 只对每次预测的最后一个token的logits进行softmax操作，然后计算具体的token索引
    5. 将最后一个预测的索引加入到解码器初始输入序列的后面，序列加长
    6. 继续预测，继续放到解码器输入序列的后面，知道预测的toekn是结束token或者得到最长序列位置
    7. 返回最后一次的预测序列。

    Arguments:
        model: torch.nn.Module, 模型
        src: Tensor, shape ``[src_seq_len, batch_size]``源语言张量
        src_mask：Tensor, shape ``[src_seq_len, src_seq_len]``未来位置屏蔽掩码，一般情况不需要掩码
        max_len: int, 翻译文本的最大长度
        start_symbol：int, 开始token，取决于模型训练数据设置的开始token的索引

    Returns:
        ys, Tensor, shape ``[src_seq_len, 1]``
        预测序列
    """


    src = src.to(DEVICE)
    src_mask = src_mask.to(DEVICE)
    # 编码器提取的特征
    #预测过程没有填充操作
    memory = model.encode(src, src_mask)
    #目标输入的第一个token，自己作为一个序列:batch和sequence都是1
    ys = torch.ones(1, 1).fill_(start_symbol).type(torch.long).to(DEVICE)
    for i in range(max_len-1):
        memory = memory.to(DEVICE)
        #预测阶段依然要屏蔽未来位置
        tgt_mask = (generate_square_subsequent_mask(ys.size(0))
                    .type(torch.bool)).to(DEVICE)
        #形状：(sequence, batch, emb_size)
        out = model.decode(ys, memory, tgt_mask)
        out = out.transpose(0, 1)
        #只对预测的最后一个token求logits，形状：(batch,vocab_size)
        prob = model.generator(out[:, -1])
        #预测的最后一个token的索引
        _, next_word = torch.max(prob, dim=1)
        next_word = next_word.item()
        #在后面添加预测的索引，作为新的输入
        ys = torch.cat([ys,
                        torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=0)
        if next_word == EOS_IDX:
            break
    return ys


# 翻译
def translate(model: torch.nn.Module, src_sentence: str):
    model.eval()
    #文本token化，词典输出indice,添加开始和结束token
    #src形状：(sequence,1)
    src = text_transform[SRC_LANGUAGE](src_sentence).view(-1, 1)
    num_tokens = src.shape[0]
    #不需要掩码，所以里面全是False
    src_mask = (torch.zeros(num_tokens, num_tokens)).type(torch.bool)
    tgt_tokens = greedy_decode(model,  src, src_mask, max_len=num_tokens + 5, start_symbol=BOS_IDX).flatten()

    return " ".join(vocab_transform[TGT_LANGUAGE].lookup_tokens(list(tgt_tokens.cpu().numpy()))).replace("<bos>", "").replace("<eos>", "")

In [8]:
print(translate(transformer, "Eine Gruppe von Menschen steht vor einem Iglu ."))

 A group of people stand in front of an igloo 
